In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

In [23]:
# Parametry
batch_size = 64
learning_rate = 0.001
num_epochs = 20
patience = 3  # Liczba epok bez poprawy na zbiorze walidacyjnym przed zatrzymaniem

# Transformacje danych
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Pobieranie i dzielenie danych na treningowe i walidacyjne
dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [15]:
class SimpleFCNN(nn.Module):
    def __init__(self):
        super(SimpleFCNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)  # spłaszczenie obrazu
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Inicjalizacja modelu, kryterium i optymalizatora
model = SimpleFCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [17]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    best_val_loss = np.inf  # najlepsza dotychczasowa strata walidacyjna
    epochs_no_improve = 0  # licznik epok bez poprawy

    for epoch in range(num_epochs):
        # Trenowanie
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        # Walidacja
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        
        # Wyświetlanie wyników
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping: sprawdzenie poprawy na zbiorze walidacyjnym
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()  # zapisywanie najlepszego modelu
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping triggered")
            model.load_state_dict(best_model_state)  # ładowanie najlepszego modelu
            break

    return model


In [19]:
trained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)

Epoch [1/20], Train Loss: 0.4176, Val Loss: 0.2354
Epoch [2/20], Train Loss: 0.2005, Val Loss: 0.1761
Epoch [3/20], Train Loss: 0.1491, Val Loss: 0.1409
Epoch [4/20], Train Loss: 0.1217, Val Loss: 0.1515
Epoch [5/20], Train Loss: 0.1017, Val Loss: 0.1274
Epoch [6/20], Train Loss: 0.0901, Val Loss: 0.1015
Epoch [7/20], Train Loss: 0.0764, Val Loss: 0.1110
Epoch [8/20], Train Loss: 0.0684, Val Loss: 0.0932
Epoch [9/20], Train Loss: 0.0600, Val Loss: 0.1040
Epoch [10/20], Train Loss: 0.0560, Val Loss: 0.1116
Epoch [11/20], Train Loss: 0.0529, Val Loss: 0.1101
Epoch [12/20], Train Loss: 0.0468, Val Loss: 0.1139
Epoch [13/20], Train Loss: 0.0429, Val Loss: 0.1121
Epoch [14/20], Train Loss: 0.0403, Val Loss: 0.1145
Epoch [15/20], Train Loss: 0.0362, Val Loss: 0.1206
Epoch [16/20], Train Loss: 0.0366, Val Loss: 0.1145
Epoch [17/20], Train Loss: 0.0301, Val Loss: 0.1465
Epoch [18/20], Train Loss: 0.0345, Val Loss: 0.1024
Epoch [19/20], Train Loss: 0.0290, Val Loss: 0.1309
Epoch [20/20], Train 

In [21]:
# Pobieranie danych testowych
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Ewaluacja na zbiorze testowym
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Sprawdzenie poprawnych przewidywań
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        
test_loss /= len(test_loader)
accuracy = correct / len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%")


Test Loss: 0.0935, Test Accuracy: 97.64%
